In [1]:
import pandas as pd
from flashtext import KeywordProcessor
from tqdm import tqdm
tqdm.pandas()
import spacy
import sys
import re
import nltk
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)

#for downloading spacy stuff
#!{sys.executable} -m spacy download en_core_web_sm

#nlp = spacy.load("en_core_web_lg")

import stanza

from tqdm import tqdm
tqdm.pandas()

nlp2 = stanza.Pipeline(lang='en', processors='tokenize,ner')

2021-06-07 08:23:43 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2021-06-07 08:23:43 INFO: Use device: cpu
2021-06-07 08:23:43 INFO: Loading: tokenize
2021-06-07 08:23:43 INFO: Loading: ner
2021-06-07 08:23:44 INFO: Done loading processors!


In [2]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities.engine import RecognizerResult, OperatorConfig

# Set up the engine, loads the NLP module (spaCy model by default) and other PII recognizers
analyzer = AnalyzerEngine()

# Initialize the engine with logger.
engine = AnonymizerEngine()

def presidio_tagger(text):
    
    tags = analyzer.analyze(text=text,
                       entities=["PHONE_NUMBER",'EMAIL_ADDRESS','PERSON'],
                       language='en')
    
    result = engine.anonymize(
    text=text,
    analyzer_results=tags,
    operators={"PERSON": OperatorConfig("replace", {"new_value": "<REMOVE>"}),
              "PHONE_NUMBER": OperatorConfig("replace", {"new_value": "<REMOVE>"}),
              "EMAIL_ADDRESS": OperatorConfig("replace", {"new_value": "<REMOVE>"})})
    
    return result.text
    

In [3]:
errors = []
def name_remover_stanza(text):
    
    try:
        doc2 = nlp2(text)

        sentences =[]
        for sentence in doc2.sentences:
            for token in sentence.tokens:
                #Removing all tokens which have been tagged as PERSON
                if 'PERSON' not in token.ner:
                    sentences.append(token.text)
            #Adding new line symbol after each sentence to clearly denote the end of the sentence
            sentences.append('\n')
        listToStr = ' '.join([str(elem) for elem in sentences]) 
        return listToStr
    except:
        #Catching any errors and storing those values in errors
        print('error_found')
        errors.append(text)
        return text

In [ ]:
# df = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Job vacancies/emscad_v1 2.csv")

# #Removing the html tags in the columns.
# rows = []
# for t in tqdm(df['description']):
#     soup = BeautifulSoup(t,"lxml")
#     rows.append(soup.get_text())
# df['description'] = rows

# rows = []
# #Some columns contain nan therefore appending no text
# for t in tqdm(df['requirements']):
#     if str(t) == 'nan':
#         rows.append('')
#     else:
#         soup = BeautifulSoup(t,"lxml")
#         rows.append(soup.get_text())
# df['requirements'] = rows

# df['job_description'] = df['description'] + ' ' + df['requirements']

In [ ]:
df = pd.read_csv('/Users/ivowings/Sync/Thesis/Datasources/Job vacancies/emscad_v1_anonymized_complete.csv',sep=',')
df = df.fillna('no value')

from datetime import datetime



for column in tqdm(df.columns):
    df[column] = df[column].str.replace('#[\w-]+#', ' ',regex=True)
    df[column] = df[column].str.replace('#[\w\s-]+#', ' ',regex=True)
    df[column] = df[column].str.replace('^(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]\d{3}[\s.-]\d{4}$', ' ',regex=True)
    df[column] = df[column].apply(presidio_tagger)
    df[column] = df[column].str.replace('<REMOVE>', ' ',regex=False)
    df[column] = df[column].str.replace(r'  +', ' ',regex=True)
    print('Starting Stanza at: ',datetime.now().time())
    df[column] = df[column].apply(name_remover_stanza)
    

  0%|          | 0/19 [00:00<?, ?it/s]

Starting Stanza at:  08:25:55.936629


  5%|▌         | 1/19 [24:34<7:22:16, 1474.25s/it]

Starting Stanza at:  08:50:11.830656


 11%|█         | 2/19 [41:57<5:45:51, 1220.67s/it]

...

Starting Stanza at:  09:32:54.380780


 26%|██▋       | 5/19 [7:00:36<32:07:22, 8260.15s/it]

Starting Stanza at:  15:33:28.642450


 32%|███▏      | 6/19 [15:57:46<59:15:25, 16409.67s/it]

Starting Stanza at:  00:25:55.836956


In [ ]:
df.to_csv('/Users/ivowings/Desktop/emscad_presidio.csv',index=False)

In [ ]:
df.head()

In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
test = pd.read_csv('/Users/ivowings/Sync/Thesis/Datasources/Job vacancies/emscad_v1_anonymized_complete.csv',sep=',')

# test = test[['job_description','fraudulent']]
test['job_description'] = test['job_description'].str.replace('#[\w-]+#', ' ',regex=True)
test['job_description'] = test['job_description'].str.replace('#[\w\s-]+#', ' ',regex=True)
test['job_description'] = test['job_description'].str.replace('^(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]\d{3}[\s.-]\d{4}$', ' ',regex=True)
test['job_description'] = test['job_description'].progress_apply(presidio_tagger)
# test.head()
# test.to_csv('/Users/ivowings/Sync/Thesis/Datasources/Job vacancies/emscad_v1_anonymized.csv',index=False)

In [ ]:
# df['job_description'] = df['job_description'].progress_apply(name_remover)
# df.to_csv("insert path", index=False)